In [1]:
import numpy as np
%run NTRUencrypt.ipynb
%run NTRUdecrypt.ipynb
%run NTRUutil.ipynb

import sys
from os.path import exists

def bincipher(s,q=64):
    maxb=1+int(np.ceil(np.log2(q)))
    l=s.split(" ")
    outstr=""
    for k in l:
        if len(k)>0:
            b=bin(int(k))
            if b[0]=="-":
                r=b[3:]+"1"
            else:
                r=b[2:]+"0"
            outstr+="0"*(maxb-len(r))+r
    
    return outstr
    
def debincipher(c,q):
    maxb=1+int(np.ceil(np.log2(q)))
    valarr=[str((1-2*int(c[i+maxb-1]))*int(c[i:i+maxb-1],2)) for i in range(0,len(c),maxb)]
    return " ".join(valarr)
    
def keygen(secarg="-M",filename="key"):
    N1 = NTRUdecrypt()

    # Set the parameters based on input flags (or lack thereof)
    if (secarg=="-M"):
        N1.setNpq(N=107,p=3,q=64,df=15,dg=12,d=5)
    elif (secarg=="-HH"):
        N1.setNpq(N=503,p=3,q=256,df=216,dg=72,d=55)
    else:
        N1.setNpq(N=167,p=3,q=128,df=61,dg=20,d=18)
        
    # Actually generate and save the public and private keys
    N1.genPubPriv(filename)

def encrypt(message="Hello world", keyfile="key"):
    
    # First check if the public key file exists
        if not exists(keyfile+".pub"):
            sys.exit("ERROR : Public key '"+keyfile+".pub' not found.")

        # Then initialise an encryption class
        E = NTRUencrypt()

        # And read the public key
        E.readPub(keyfile+".pub")

        # Then encrypt the data
        E.encryptString(message)
        
        return bincipher(E.Me,E.q)

def decrypt(cipher, keyfile="key"):

    # First check if the private key file exists
    if not exists(keyfile+".priv"):
        sys.exit("ERROR : Public key '"+keyfile+".priv' not found.")

    # Then initialise an decryption class
    D = NTRUdecrypt()

    # And read the private key
    D.readPriv(keyfile+".priv")
    
    # Then decrypt the string
    D.decryptString(debincipher(cipher,D.q))
    
    return D.M